In [1]:
import pandas
df = pandas.read_excel(open('06222016 Staph Array Data.xlsx','rb'), sheetname='Plate 1')
df.head()

,DataType:,Avg Net MFI,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53
0,Sample ID,Hospital,Age,Gender,PSMalpha2,PSMalpha3,psmalpah4,BSA,Betatoxin,hIgA,...,Exoprotein ext,Rabbit IgG,LukF-PV,PSM 4variant,PC4,PNAG,PC16,HLA -2,Tetanus Toxoid,Tetanus Toxoid
1,Standard 10,NaN,NaN,NaN,3480,1678,2416,NaN,9189,11785,...,9250,3171,5303,1491,237,1362,1162,13300,14945,NaN
2,Standard 100,NaN,NaN,NaN,1811,1534,1342,NaN,19049,6289,...,9587,2347,18966,966,205,1175,2103,16852,18060,NaN
3,Standard 1000,NaN,NaN,NaN,823,1073,677,NaN,20730,1445,...,6598,2707,27274,598,47,737,744,12894,15435,NaN
4,Standard 10000,NaN,NaN,NaN,319,438,262,NaN,7920,671,...,2002,2256,24868,251,25,279,146,4648,4918,NaN


In [6]:
sampleID=df.ix[:,0]
print(sampleID)

0             Sample ID
1           Standard 10
2          Standard 100
3         Standard 1000
4        Standard 10000
5       Standard 100000
6      Standard 1000000
7     Standard 10000000
8          23234 V1 100
9         23234 V1 1000
10      23234 V1 10000 
11      23234 V1 100000
12         23234 V2 100
13        23234 V2 1000
14       23234 V2 10000
15      23234 V2 100000
16         28531 V1 100
17        28531 V1 1000
18       28531 V1 10000
19      28531 V1 100000
20         28531 V2 100
21        28531 V2 1000
22       28531 V2 10000
23      28531 V2 100000
24         28729 V1 100
25        28729 V1 1000
26       28729 V1 10000
27      28729 V1 100000
28         28729 V2 100
29        28729 V2 1000
30       28729 V2 10000
31      28729 V2 100000
32         28729 V3 100
33        28729 V3 1000
34       28729 V3 10000
35      28729 V3 100000
36         33142 V1 100
37        33142 V1 1000
38       33142 V1 10000
39      33142 V1 100000
40         33142 V2 100
41        33142 